In [1]:
import numpy as np # linear algebra
!pip install datasets --quiet
import pandas as pd
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install langchain

In [3]:
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

2024-03-17 01:40:28.901313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 01:40:28.901431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 01:40:29.032510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"

!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

--2024-03-17 01:40:40--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.03s   

2024-03-17 01:40:41 (63.9 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-03-17 01:40:42--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [5]:
train, trial = df = pd.read_table('train.tsv'), pd.read_table('trial.tsv')

In [6]:
train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)
trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)

In [7]:
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

In [8]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype = torch.float16, trust_remote_code = True, device_map = "auto")

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [17]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

generation_config.max_new_tokens = 500 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.6 # randomness of the generated tex
generation_config.typical_p = 0.5 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)
llm = HuggingFacePipeline(pipeline = pipe)

In [18]:
from datasets import Dataset
data = Dataset.from_pandas(trial.iloc[:35])

In [19]:
from transformers import set_seed
from tqdm import tqdm

In [23]:
set_seed(9)
set_seed(9)
set_seed(9)
set_seed(9)
set_seed(9)

In [30]:
all_results = []
all_preds = []

In [31]:
import numpy as np
from scipy.stats import pearsonr
import re
import warnings
warnings.filterwarnings('ignore')

# Hypothetical setup - replace with your actual implementations
set_seed(9)  # Define or import your set_seed function

# Define your hyperparameter space
temperatures = np.arange(0.4, 1.0, 0.05)  # Adjust the step size as needed
typical_ps = np.arange(0.5, 0.9, 0.05)  # Define this range as per your hypothesis

# Fixed hyperparameters
max_new_tokens = 500

# Iterate over your hyperparameters
for temp in temperatures:
    for tp in typical_ps:
        # Setup the model with current hyperparameters
        generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
        generation_config.max_new_tokens = max_new_tokens
        generation_config.temperature = temp
        generation_config.typical_p = tp
        generation_config.do_sample = True

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            return_full_text=True,
            generation_config=generation_config,
        )
        llm = HuggingFacePipeline(pipeline=pipe)

        # Initialize containers for this iteration's results
        predictions = []
        results = []

        # Loop through your data and make predictions
        for row in data:
            torch.cuda.empty_cache()
            query = f"[INST]Estimate the lexical complexity of the token in the context, give one single float value between 0 and 1 " \
                    f"where 0 means the lowest lexical complexity and 1 means the highest lexical complexity (very complex). " \
                    f"the third quartile at 0.371 meaning most of the tokens complexity below 0.371."\
                    f"use all possible features to accuratly estimate it:"\
                    f"The Context: {row['sentence']}"\
                    f"Token: {row['token']}[/INST]"

            result = llm(query).strip()  # Adjust according to your function's return value
            predictions.append(result)
            match = re.search(r"[-+]?\d*\.\d+|\d+", result)
            if match:
                number = float(match.group(0))
            else:
                number = None
            results.append(number)

        # Compute Pearson correlation for this set of results
        # Assuming 'actual_values' is a list of actual numbers to compare against
        pearson_correlation_value, _ = pearsonr(results, trial["complexity"])  # Replace 'actual_values' with your actual data
        all_results.append(results)
        all_preds.append(predictions)

        # Print the results for this iteration
        print(f"Configuration: Temperature={temp}, Typical_p={tp}, PearsonR={pearson_correlation_value}")
        print('\n\n\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Configuration: Temperature=0.45, Typical_p=0.55, PearsonR=0.5333845162321577






Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Configuration: Temperature=0.45, Typical_p=0.6000000000000001, PearsonR=0.3644643929829886






Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

...

Configuration: Temperature=0.6499999999999999, Typical_p=0.6500000000000001, PearsonR=0.340988395896055






Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

Configuration: Temperature=0.6499999999999999, Typical_p=0.7000000000000002, PearsonR=0.332026246667946






Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [34]:
with open("/kaggle/working/results.txt", 'w') as f:
    for item in all_results:
        f.write("%s\n" % item)

with open("/kaggle/working/text.txt", 'w') as f:
    for item in all_preds:
        f.write("%s\n\n\n" % item)

In [44]:
for i in all_results:
# trial = trial[:35]
    print(pearsonr(i, trial["complexity"]))

PearsonRResult(statistic=0.4061502661877224, pvalue=0.015480629395990703)
PearsonRResult(statistic=0.5163926764280076, pvalue=0.001494815888883149)
PearsonRResult(statistic=0.36465687057466056, pvalue=0.0312582162814451)
PearsonRResult(statistic=0.3306622099276742, pvalue=0.05236187571539662)
PearsonRResult(statistic=0.41454469910206554, pvalue=0.013288199520706057)
PearsonRResult(statistic=0.2491507064199073, pvalue=0.1489250870494208)
PearsonRResult(statistic=0.3874401614090028, pvalue=0.02147541692389387)
PearsonRResult(statistic=0.30232486831130834, pvalue=0.07753171865904158)
PearsonRResult(statistic=0.4230840208084625, pvalue=0.01133244603051881)
PearsonRResult(statistic=0.5333845162321577, pvalue=0.0009689130957664989)
PearsonRResult(statistic=0.3644643929829886, pvalue=0.031354228275597154)
PearsonRResult(statistic=0.46461788566203155, pvalue=0.004925063941180165)
PearsonRResult(statistic=0.36780005815794165, pvalue=0.02972406214948039)
PearsonRResult(statistic=0.35198168300327

In [48]:
print(pearsonr(all_results[32], trial["complexity"]))

PearsonRResult(statistic=0.5959506989889801, pvalue=0.00015845143223100648)


In [49]:
best_result = all_results[32]
best_preds = all_preds[32]

In [52]:
with open("/kaggle/working/best_result.txt", 'w') as f:
    for item in best_result:
        f.write("%s\n" % item)

with open("/kaggle/working/best_preds.txt", 'w') as f:
    for item in best_preds:
        f.write("%s\n\n\n" % item)

In [54]:
print(pearsonr(best_result, trial["complexity"]))

PearsonRResult(statistic=0.6035674420921041, pvalue=0.00012383496569887012)
